In [ ]:
import numpy as np
from colors import *

In [ ]:
SCORING = {
    "SINGLE": 100,
    "DOUBLE": 300,
    "TRIPLE": 500,
    "QUAD": 800,
    "TSPIN_MINI": 100,
    "TSPIN": 400,
    "TSPIN_MINI_SINGLE": 200,
    "TSPIN_SINGLE": 800,
    "TSPIN_MINI_DOUBLE": 400,
    "TSPIN_DOUBLE": 1200,
    "TSPIN_TRIPLE": 1600,
    "TSPIN_QUAD": 2600,
    "BACKTOBACK_MULTIPLIER": 1.5,
    "COMBO": 50,
    "ALL_CLEAR": 3500,
    "SOFTDROP": 1,
    "HARDDROP": 2
}

In [ ]:
class PieceData():
    def __init__(self, data, num_rot, start_x_position):
        self.num_rot = num_rot
        self.start_x_position = start_x_position
        
        self.data = data
        
        self.margins = []
        for rot in range(num_rot):
            npdata = np.array(data[rot])
            
            left = npdata[:, 1].min()
            right = npdata[:, 1].max()
        
            self.margins.append((left, right))
        
        # self.width = []
        # self.height = []
        # for i in range(num_rot):
        #     cells=np.rot90(data, i)
        #     self.data.append(cells)
        #     self.width.append(np.argwhere((cells == 1).any(axis=0)).max() + 1)
        #     self.height.append(np.argwhere((cells == 1).any(axis=1)).max() + 1)
        # # self.data = np.array(self.data)

    
    def __str__(self):
        return '\n'.join(map(lambda x: str(x), self.data))
    def __repr__(self):
        return self.__str__()
        
        
piece_data = []

# 0: t-piece
piece_data.append(PieceData([((0, 1), (1, 0), (1, 1), (1, 2)),((0, 1), (1, 1), (1, 2), (2, 1)),((1, 0), (1, 1), (1, 2), (2, 1)),((0, 1), (1, 0), (1, 1), (2, 1))], 4, 3))
# 1: l-piece
piece_data.append(PieceData([((0, 2), (1, 0), (1, 1), (1, 2)),((0, 1), (1, 1), (2, 1), (2, 2)),((1, 0), (1, 1), (1, 2), (2, 0)),((0, 0), (0, 1), (1, 1), (2, 1))], 4, 3))
# 2: j-piece
piece_data.append(PieceData([((0, 0), (1, 0), (1, 1), (1, 2)),((0, 1), (0, 2), (1, 1), (2, 1)),((1, 0), (1, 1), (1, 2), (2, 2)),((0, 1), (1, 1), (2, 0), (2, 1))], 4, 3))
# 3: o-piece
piece_data.append(PieceData([((0, 1), (0, 2), (1, 1), (1, 2)),((0, 1), (0, 2), (1, 1), (1, 2)),((0, 1), (0, 2), (1, 1), (1, 2)),((0, 1), (0, 2), (1, 1),(1, 2))], 1, 4))
# 4: i-piece
piece_data.append(PieceData([((1, 0), (1, 1), (1, 2), (1, 3)), ((0, 2), (1, 2), (2, 2), (3, 2)), ((2, 0), (2, 1), (2, 2), (2, 3)),((0, 1), (1, 1), (2, 1), (3, 1))], 4, 3))
# 5: s-piece
piece_data.append(PieceData([((0, 1), (0, 2), (1, 0), (1, 1)),((0, 1), (1, 1), (1, 2), (2, 2)),((1, 1), (1, 2), (2, 0), (2, 1)),((0, 0), (1, 0), (1, 1), (2, 1))], 4, 3))
# 6: z-piece
piece_data.append(PieceData([((0, 0), (0, 1), (1, 1), (1, 2)),((0, 2), (1, 1), (1, 2), (2, 1)),((1, 0), (1, 1), (2, 1), (2, 2)),((0, 1), (1, 0), (1, 1), (2, 0)),], 4, 3))


kicks = {
    # (kick_from, kick_to): (offset1, offset2, offset3, offset4)
    (0, 1): ((+0, -1), (-1, -1), (+2, +0), (+2, -1)),  # 0 -> R | CW
    (0, 3): ((+0, +1), (-1, +1), (+2, +0), (+2, +1)),  # 0 -> L | CCW
    (1, 0): ((+0, +1), (+1, +1), (-2, +0), (-2, +1)),  # R -> 0 | CCW
    (1, 2): ((+0, +1), (+1, +1), (-2, +0), (-2, +1)),  # R -> 2 | CW
    (2, 1): ((+0, -1), (-1, -1), (+2, +0), (+2, -1)),  # 2 -> R | CCW
    (2, 3): ((+0, +1), (-1, +1), (+2, +0), (+2, +1)),  # 2 -> L | CW
    (3, 0): ((+0, -1), (+1, -1), (-2, +0), (-2, -1)),  # L -> 0 | CW
    (3, 2): ((+0, -1), (+1, -1), (-2, +0), (-2, -1)),  # L -> 2 | CCW
}

i_kicks = {
    (0, 1): ((+0, -2), (+0, +1), (+1, -2), (-2, +1)),  # 0 -> R | CW
    (0, 3): ((+0, -1), (+0, +2), (-2, -1), (+1, +2)),  # 0 -> L | CCW
    (1, 0): ((+0, +2), (+0, -1), (-1, +2), (+2, -1)),  # R -> 0 | CCW
    (1, 2): ((+0, -1), (+0, +2), (-2, -1), (+1, +2)),  # R -> 2 | CW
    (2, 1): ((+0, +1), (+0, -2), (+2, +1), (-1, +2)),  # 2 -> R | CCW
    (2, 3): ((+0, +2), (+0, -1), (-1, +2), (+2, -1)),  # 2 -> L | CW
    (3, 0): ((+0, +1), (+0, -2), (+2, +1), (-1, -2)),  # L -> 0 | CW
    (3, 2): ((+0, -2), (+0, +1), (+1, -2), (-2, +1)),  # L -> 2 | CCW
}

In [ ]:
class Piece():
    def __init__(self, ptype, pos=None):
        self.pos = pos if pos is not None else [0, piece_data[ptype].start_x_position]
        self.ptype = ptype
        self.pdata = piece_data[ptype]
    
    def __str__(self):
        return str(self.pdata)
    def __repr__(self):
        return self.__str__()
    
    def clone(self, ptype=None, pos=None):
        if ptype is None:
            ptype = self.ptype
        if pos is None:
            pos = self.pos
        
        return Piece(ptype, pos)
        
# def drop_piece_on_field(P, F):

p1 = Piece(4)
p1.pdata.data[3]


# for i, pd in enumerate(p1.pdata.data):
#     print(i)
#     print(pd)
#     print()

# p2 = Piece(0)

# id(p1.pdata) == id(p2.pdata)

In [352]:
class Field():
    def __init__(self, shape=(20, 10), data=None):
        if data is None:
            self.field_data = np.zeros(shape)
        else:
            self.field_data = data

        self.width = shape[1]
        self.height = shape[0]
        
        
    def _extract_piece_info(self, piece, rot, pos=None):
        px = piece.pos[0] if pos is None else pos[0]
        py = piece.pos[1] if pos is None else pos[1]
        
        return (px, py)
    
        
    def is_collision(self, piece, rot, pos=None):
        """ Returns whether the given piece collides with the current field. """
        # check if collision between shape and existing play field
        # if 'pos' is not passed, it will use the piece's position
        (px, py) = self._extract_piece_info(piece, rot, pos)
        
        for x, y in piece.pdata.data[rot]:
            if x + px not in range(self.field_data.shape[0]):
                return True

            if y + py not in range(self.field_data.shape[1]):
                return True

            if self.field_data[x + px, y + py] != 0:
                return True

        return False
    
        
    def get_follow_states(self, piece):
        """ Generates all possible next states that follow from the current state of the field. """
        next_states = []
        
        # rotate the piece in all possible directions
        for rot in range(piece.pdata.num_rot):            
            (ml, mr) = piece.pdata.margins[rot]
                        
            # move the piece to all possible positions
            # for px in range(width_diff + 1):
            for py in range(-ml, self.width - mr): 
                px = self.drop_piece(piece, rot, [0, py])
                
                dropped_pos = [px, py]
                
                # print(f'DROPPED {piece.pdata.data[rot]} TO [{py}, {px}] WITH ROT = {rot}')
                # self._print_board(self._add_piece_to_board(piece, rot, dropped_pos))
                # new_state = self._add_piece_to_board(piece, rot, dropped_pos)
                # if not next((True for elem in next_states if np.array_equal(elem, new_state)), False):
                #     next_states.append(new_state)
                # # else:
                # #     print('REMOVED DUPLIUCATE')
                next_states.append(self._add_piece_to_board(piece, rot, dropped_pos))
                
                # try kick/flip with all possible rotations
                self._kick_expansion(piece, rot, dropped_pos, next_states)
                
                
                
                left = 1
                while True:
                    if self.is_collision(piece, rot, [px, py - left]) or not self.is_collision(piece, rot, [px - 1, py - left]):
                        break
                    else:
                        left += 1
                        self._kick_expansion(piece, rot, [px, py - left], next_states)
                
                right = 1
                while True:
                    if self.is_collision(piece, rot, [px, py + right]) or not self.is_collision(piece, rot, [px - 1, py + right]):
                        break
                    else:
                        right += 1
                        self._kick_expansion(piece, rot, [px, py + right], next_states)

                        
                
                # new_field = self.drop_piece_on_field(piece, rot, [0, px])
                # next_states.append(Field(data=new_field))
        
        # remove duplicates
        next_states = np.unique(np.array(next_states).reshape(len(next_states), -1), axis=0).reshape((-1, 20, 10))
        
        return next_states
    
    def _kick_expansion(self, piece, rot, dropped_pos, states):
        (px, py) = self._extract_piece_info(piece, rot, dropped_pos)

        for kick_rot in range(piece.pdata.num_rot):
            (kick_succ, kick_result) = self._rotate(piece, rot, rot - kick_rot, dropped_pos)

            if kick_succ and self.is_collision(piece, kick_result[2], [px - 1, py]):
                # print(f'SUCESSFULLY KICKFLIPPED: {kick_result}')

                # self._print_board(self.drop_piece_on_field(piece, kick_result[2], [kick_result[0], kick_result[1]])) 

                # possible state
                # new_state = self.drop_piece_on_field(piece, kick_result[2], [kick_result[0], kick_result[1]])
                # if not next((True for elem in next_states if np.array_equal(elem, new_state)), False):
                #     next_states.append(new_state)
                # else:
                #     print('REMOVED DUPLIUCATE')
                states.append(self.drop_piece_on_field(piece, kick_result[2], [kick_result[0], kick_result[1]]))
    
    
    def _rotate(self, piece, rot, rot_offset, pos=None):
        """Tries to rotate the given piece (applying kicks/spins, if else not possible) and returns if the rotation worked and the new piece infos, as a tuple."""
        (px, py) = self._extract_piece_info(piece, rot, pos)
        to_rot = (rot + rot_offset) % piece.pdata.num_rot
        
        # if it does not collide, just rotate it
        if not self.is_collision(piece, to_rot, pos):
            return (True, (px, py, to_rot))
        
        # otherwise try to kick it
        table = i_kicks if piece.ptype == 4 else kicks
        
        return self._kick_piece(table, piece, rot, to_rot, pos)
        
        
    def _kick_piece(self, table, piece, rot, to_rot, pos=None):
        """Tries to kick the piece and returns if it was successful and the new piece infos, as a tuple."""
        (px, py) = self._extract_piece_info(piece, rot, pos)
        
        if not (rot, to_rot) in table:
            return (False, None)
        
        for x, y in table[rot, to_rot]:
            # for each offset, test if it's valid
            if not self.is_collision(piece, to_rot, [px + x, py + y]):
                # if it's vlaid, kick it and break
                return (True, (px + x, py + y, to_rot))
        
        return (False, None)
        
    
    def drop_piece(self, piece, rot, pos=None):
        """Tries to drop the given piece on this field and returns the height it landed on."""
        (px, py) = self._extract_piece_info(piece, rot, pos)
        
        while not self.is_collision(piece, rot, [px, py]):
            px += 1

        return px - 1
    
    def _add_piece_to_board(self, piece, rot, pos=None):
        """Places the piece on the board and returns the new resulting field."""
        (px, py) = self._extract_piece_info(piece, rot, pos)
        
        temp_field = np.copy(self.field_data)
        
        for x, y in piece.pdata.data[rot]:
            temp_field[x + px, y + py] = 1
        
        return temp_field
    
    def drop_piece_on_field(self, piece, rot, pos=None):
        """ Drops the given piece on this field, returning the newly generated field."""
        (px, py) = self._extract_piece_info(piece, rot, pos)
        
        px = self.drop_piece(piece, rot, pos)
        
        return self._add_piece_to_board(piece, rot, [px, py])
    
    
    def drop_piece_on_field_(self, piece, rot, pos=None):
        """ Drops the given piece on this field in-place."""
        self.field_data = self.drop_piece_on_field(piece, rot, pos)
    
    def _print_board(self, board):
        for row in board:
            print(''.join(map(lambda cell: ' ' if cell == 0 else '█', row)))
    
    def __str__(self):
        return str(self.field_data)
    
    def __repr__(self):
        return self.__str__()

In [359]:
f = Field()
# --- t-spin triple ---
f.field_data[14,:5] = 1

f.field_data[15,:4] = 1

f.field_data[16,:4] = 1
f.field_data[16,5:] = 1

f.field_data[17,0:4] = 1
f.field_data[17,6:] = 1

f.field_data[18:, :] = 1
f.field_data[18,4] = 0


f.field_data[14,:5] = 1

f.field_data[15,:4] = 1


# --- t-spin double ---
# f.field_data[16,:3] = 1
# f.field_data[16,5:] = 1

# f.field_data[17,0:3] = 1
# f.field_data[17,6:] = 1

# f.field_data[18:, :] = 1
# f.field_data[18,4] = 0


f._print_board(f.field_data)

p = Piece(1, [0,0])

f._print_board((f._add_piece_to_board(p, 0, [9,0])))

# print(f._rotate(p, 3, 1, [9,0]))

# f._print_board((f._add_piece_to_board(p, 0, [9,0])))

# print(f.drop_piece(p, 0, [0, 1]))

# print(f.drop_piece_on_field(p, 0, [14,4]))

# f.is_collision(p, 0, [16,1])

# %timeit f._get_piece_margin(p, 0)
fs= f.get_follow_states(p)

# f.drop_piece_on_field_(p, 0)
# f.field_data[-1]

# f.field_data[1,1]

# p2 = Piece(4)
# f.get_follow_states(p2)


# f.is_collision(p, 2, [9,8])




for s in fs:
    print(f._print_board(s))

          
          
          
          
          
          
          
          
          
          
          
          
          
          
█████     
████      
████ █████
████  ████
████ █████
██████████
          
          
          
          
          
          
          
          
          
  █       
███       
          
          
          
█████     
████      
████ █████
████  ████
████ █████
██████████
5.67 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
          
          
          
          
          
          
          
          
          
          
          
          
          
          
█████    █
████   ███
████ █████
████  ████
████ █████
██████████
None
          
          
          
          
          
          
          
          
          
          
          
          
          
          
█████   █ 
████  ███ 
████ █████
████  ████
████ █████
██████████
None
          
          
          
      

In [ ]:
# %%timeit

for i, s in enumerate(fs):
    # print(f._print_board(s))
    for j in range(i):
        # print(i,j)
        if np.array_equal(s, fs[j]):
            pass
            print(f'found equal,  {i} and {j}')

In [ ]:
%timeit np.array_equal(fs[0], fs[10])

In [ ]:
# npdata = np.array(fs)

%timeit np.unique(np.array(fs).reshape(len(fs), -1), axis=0)

In [ ]:
last = fs[0]
rest = fs[1:]

def is_arr_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if elem is myarr), False)

def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if np.array_equal(elem, myarr)), False)
    # return np.any(np.all(myarr == list_arrays, axis=1))

In [ ]:
%timeit arreq_in_list(last, rest)

%timeit np.multiply(a, b)
1.65 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.67 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

%timeit a * b
1.64 µs ± 47.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.65 µs ± 39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


%timeit (np.add(a, b))
1.46 µs ± 68.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.47 µs ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

%timeit (np.count_nonzero(a*b))
2.99 µs ± 88.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.22 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.91 µs ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


